# Lecture03 

논문 제목 : Convolutional Neural Networks for Sentence Classification(Yoon Kim, 2014)  
논문 출처 : https://arxiv.org/abs/1408.5882

### Classification 
이번시간에는 Convolution operator를 NLP에 사용하여 의미있는 결과를 가지오 왔던 Yoon Kim박사님의 논문을 Implementation해보는 시간을 가지겠습니다. 우선 Convolution이라는 것은 Image를 다루는 분야에서 오랫동안 사용된 operator입니다. 우리가 흔히 알고있는 Convolutional Neural Networks가 한 예이죠. Convolution을 통해서 Input data에서 의미있는 feature들을 뽑아낼 수 있습니다. **CNN**을 예로 든다면 Low-level layer에서는 데이터의 세부적인 feature들을 찾아내고 상위 레이어로 갈수록 데이터의 global한 feautre들을 학습하게 되죠. 또한 **Max-pooling** 이나 **Average-pooling**을 사용하여 데이터의 invariace한 특징들을 추출하는 방식이 보편적으로 많이 쓰였습니다. 이러한 구조를 NLP에 적용한 것이 Yoon Kim 박사님의 Text-CNN입니다. 사실 NLP에서 처음으로 적용한 모델은 아니지만 매우 간단한 모델 구조로 여러 벤치마킹 데이터에서 SoTA의 성능을 보였습니다. 그럼 지금부터 Text-CNN을 구현해 보도록 하겠습니다.

### Model Architecture
![architecture](http://i.imgur.com/JN72JHW.png)

모델의 구조는 크게 4가지의 방식을 사용합니다.  
* **CNN-rand** : 이 방식은 랜덤한 단어벡터를 사용한 구조입니다. 즉 단어에 대한 정보가 없는 상태입니다.
* **CNN-static** : 이 방식은 사전에 미리 학습된 단어벡터를 사용합니다. 하지만 이 단어자체의 fine-tuning은 이루어지지 않습니다. 
* **CNN-non-static** : 이 방식은 static과 마찬가지로 학습된 단어벡터를 사용합니다. 하지만 static과 다른점은 역전파를 통해서 input 단어들 또한 데이터셋에 따라서 fine-tuning이 된다는 점입니다.
* **CNN-multi** : 위의 모델 이미지에서 보여지는 방식입니다. 멀티 채널이 의미하는 것은 static과 non-static의 input을 merge한 상태입니다. 하지만 2개의 channel이 모두 weight update되는 것이 아니라 non-static channel만 fine-tuning하는 방식입니다.

우리는 이번시간에 가장 간단한 **CNN-rand**를 구현해보겠습니다. 데이터셋은 이전 강의에서 사용했던 **20news** 데이터를 사용합니다. 데이터를 불러오고 전처리의 과정은 최소화 하겠습니다. 또한 본 논문의 구조와 이론은 따라가되 세부 파라메터나 Trick은 이번 장에서는 제외합니다.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import re

newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

train_raw_text = newsgroups_train["data"]
train_label = newsgroups_train["target"]

test_raw_text = newsgroups_test["data"]
test_label = newsgroups_test['target']

In [ ]:
def clean_str(s):
    # only include alphanumerics
    s = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", s)
    
    # insert spaces in words with apostrophes
    s = re.sub(r"\'s", " \'s", s)
    s = re.sub(r"\'ve", " \'ve", s)
    s = re.sub(r"n\'t", " n\'t", s)
    s = re.sub(r"\'re", " \'re", s)
    s = re.sub(r"\'d", " \'d", s)
    s = re.sub(r"\'ll", " \'ll", s)
    
    # insert spaces in special characters
    s = re.sub(r",", " , ", s)
    s = re.sub(r"!", " ! ", s)
    s = re.sub(r"\(", " \( ", s)
    s = re.sub(r"\)", " \) ", s)
    s = re.sub(r"\?", " \? ", s)
    
    # only include alphanumerics again
    s = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", s)
    
    # reduce multiple spaces to single spaces
    s = re.sub(r"\s{2,}", " ", s)
    return s.strip().lower()

In [ ]:
train_clean_text = [clean_str(x) for x in train_raw_text]
test_clean_text = [clean_str(x) for x in test_raw_text]

In [ ]:
import numpy as np
y = np.concatenate([train_label, test_label])
one_hot_y = np.eye(max(y)+1)[y]

### Data information

위의 모델구조에서 볼 수 있듯이 input layer의 row는 단어들의 벡터로 이루어져 있습니다. 그렇게 되면 Document 단위의 데이터셋에서는 여러 문장으로 이루어져있기 때문에 Row의 크기가 커질 수 밖에 없겠죠. 짧은 문서의 경우에는 해당 데이터셋에서 Maximum한 문서의 길이를 이용하여 단어의 손실을 최소화 하지만 데이터의 길이의 차이가 너무 극심하거나 유의미하지 않을 때에는 평균적인 문서의 길이를 사용하는 방법도 있습니다. 우선 어떠한 방법을 사용해야하는지 알아보기 위해선 데이터의 길이별 분포를 알아야겠죠?

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [ ]:
sequence_train = [len(x.split()) for x in train_raw_text]
count_sequence = Counter(sequence_train)

key = count_sequence.keys()
value = count_sequence.values()

In [ ]:
plt.bar(key, value)

In [ ]:
max_length = max(sequence_train)
average_length = int(sum(sequence_train)/len(sequence_train))

print("Max sequence length :", max_length)
print("Average sequence length: ", average_length)

**위의 그래프를 보면 문서의 길이가 0~2000사이에 분포하고 있습니다. 하지만 가장 긴 문서의 경우 11821개의 단어들로 이루어져 있습니다. 우선 가장 긴 문서가 어떻게 이루어져 있는지 봐야겠죠.** 
문서의 길이를 선택하는 방법은 답이 정해져 있지는 않습니다. 저의 경우에는 maximum length를 2000으로 설정하겠습니다. 따라서 2000개 이상의 단어를 가지는 문서는 2000개 까지의 단어만을 사용하게 됩니다.

### Word Embedding 
이제 각각의 단어들을 index 형태로 바꾸어 주어야 합니다. Tensorflow에서는 아주 쉽게 단어들을 index로 변환해주는 함수가 있습니다. 
* **vocab_processor** : 객체생성
* **x_train** : 전처리한 문서를 단어단위로 split한 뒤 단어마다 id값을 부여한 결과 2000단어가 되지 않을 경우 id 값은 0
* **x_test** : Train데이터셋의 단어들로 이루어진 vocab을 이용하여 id값을 부여한 결과 

In [ ]:
import tensorflow as tf
from tensorflow.contrib import learn

vocab_processor = learn.preprocessing.VocabularyProcessor(average_length)
x_train = np.array(list(vocab_processor.fit_transform(train_clean_text))) 
x_test = np.array(list(vocab_processor.transform(test_clean_text)))

vocab_dict = vocab_processor.vocabulary_._mapping
vocab_size = len(vocab_dict)

In [ ]:
x_train = x_train.astype(np.int32)
one_hot_y = np.array(one_hot_y.astype(np.float32))

이제 **Placeholder**를 지정해주고 **Look-up table**을 이용하여 각 id로 매핑된 단어들에 랜덤한 벡터를 부여해줍니다.
* **placeholder x**: input x의 경우 우리는 단어의 id로 이루어진 데이터를 넣게됩니다. 따라서 shape은 **[batch size, sequence length]** 이겠죠?
* **placeholder y**: input y의 경우 class가 20개이기 때문에 **[batch size , 20]** 의 shape을 가집니다. 
* **random_word** : vocab의 단어들의 개수만큼 id가 있기 때문에 각각의 id는 어떠한 단어를 가리키게 되고 그 단어는 $d$ 차원의 벡터로 표현이 됩니다. 따라서 각각의 단어를 벡터로 표현하기 위해선 [vocab size, embedding size]의 크기가 필요하겠죠? 우리는 이번시간에 50차원을 벡터를 사용하겠습니다. 

In [ ]:
batch_size = 100
embedding_size = 50

input_x = tf.placeholder(tf.int32, [None, average_length])
input_y = tf.placeholder(tf.float32, [None, 20])

In [ ]:
random_word = tf.Variable(tf.random_normal([vocab_size, embedding_size]), dtype=tf.float32)

In [ ]:
embedded_data = tf.nn.embedding_lookup(random_word, input_x)

### Build Convolutional layer
input layer를 구성한 뒤 이제 Convolutional layer를 Build합니다. 위 논문에서는 kernel size를 다양하게 줌으로써 다양한 길이에서 feature들을 추출합니다. 쉽게 말하자면 n-gram같은 느낌입니다. 문서내에서 긍정과 부정을 판단할 때를 예로 들어 보겠습니다.   

**"나는 오늘 영화를 보았는데 매우 신선하고 재미있는 영화였다"** 

* kernel size [2, embedding size] stride **1**: "나는 오늘",  "오늘 영화를", "영화를 보았는데" $\dots$ "재미있는 영화였다"
* kernel size [2, embedding size] stride **2**: "나는 오늘",  "영화를 보았는데", "매우 신선하고", "재미있는 영화였다"

즉 여러 상황에 대한 feature 들을 생성하기 위함이고 좀 더 유연한 feutre를 추출할 수 있습니다.  
논문에서는 이 kernel size의 height를 3, 4, 5로 설정하여 사용합니다.

**Max-pooling**은 **invariance feature**를 학습시키는데 도움을 줍니다. Text 데이터일 경우에 의미있는 Paragraph 나 Context를 추출하기에 효율적인 방법입니다.
* kernel size [2, embedding size] stride **2**: "나는 오늘",  "영화를 보았는데", "매우 신선하고", "재미있는 영화였다"  

일 때 Max-pooling을 사용한다면 직관적으로는 "재미있는 영화였다"가 가장 Max값을 가질 것입니다.

### Tensorflow Function

* **tf.expand_dim** : 차원을 늘리는 용도로 사용합니다. 만약 [2, 3] 을 [2, 3, 1] 로 늘리고 싶다면 tf.expand_dim( " " , axis = -1) 
* **contrib.layers.conv2d** : weight와 bias를 직접 생성하여 사용해도 되지만 간단하게 conv2d 연산을 제공합니다.
* **tf.nn.max_pool** : max-pooling operator 입니다. Max-pool 이외에 Average-pool도 있으니 참고하세요.
* **tf.concat** : concatenate입니다. 축을기준으로 연결한다고 보시면 될 것 같네요.

In [ ]:
embedded_data_expend = tf.expand_dims(embedded_data, axis=-1)
num_filter = 100
conv_layer = []

kernel_height = [3,4,5]
for i in kernel_height:
    with tf.variable_scope("conv_kernel_%d"%i):
        conv = tf.contrib.layers.conv2d(embedded_data_expend, 100, [i, embedding_size], stride=1, padding="VALID")
        max_pool = tf.nn.max_pool(conv, ksize=[1, average_length-i+1, 1, 1], strides=[1, 1, 1, 1], padding="VALID")
        conv_layer.append(max_pool)
conv_layer = tf.concat(conv_layer, axis=-1)

In [ ]:
conv_layer = tf.reshape(conv_layer, [-1, num_filter*len(kernel_height)])

In [ ]:
with tf.variable_scope("fully_connected_layer"):
    fc1 = tf.contrib.layers.fully_connected(conv_layer, 256)
    fc2 = tf.contrib.layers.fully_connected(fc1, 512)
    fc3 = tf.contrib.layers.fully_connected(fc2, 20)

### Loss function

목적함수는 softmax cross entropy를 사용합니다. 해당 class일 확률을 계산하고 gradient를 통해서 trainable 요소들을 학습합니다. 
우리는 이 목적함수를 최소화해야 하며 AdamOptimizer를 사용합니다.

In [ ]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fc3, labels=input_y))
train_op = tf.train.AdamOptimizer(0.0001).minimize(loss)

In [ ]:
with tf.Session() as sess:
    print("Learning Start")
    sess.run(tf.global_variables_initializer())
    total_batch = int(len(x_train)/batch_size)
    for epoch in range(100):
        avg_cost = 0
        index = 0
        for step in range(total_batch):
            if (index+1)*batch_size < len(x_train):
                loss_val, _ = sess.run([loss, train_op], feed_dict={input_x:x_train[index*batch_size:(index+1)*batch_size],
                                                                    input_y:one_hot_y[index*batch_size:(index+1)*batch_size]})
                avg_cost +=loss_val
                index += 1
            else:
                loss_val, _ = sess.run([loss, train_op], feed_dict={input_x:x_train[index*batch_size:],
                                                                    input_y:one_hot_y[index*batch_size:]})
                index = 0
                avg_cost +=loss_val
        avg_cost = avg_cost/total_batch        
        print("{} Epoch Loss : {}".format(epoch,avg_cost))                 